In [14]:
import csv
import os
import re
import psycopg2
from datetime import datetime

# データベース接続情報を設定してください
DB_HOST = '54.150.30.249'
DB_PORT = '5432'
DB_NAME = 'npb'
DB_USER = 'rdsuser'
DB_PASSWORD = 'rds%5678'

# CSVファイルが保存されているディレクトリのパスを設定してください
csv_directory = 'data'  # ここを実際のパスに変更

# チーム名の正規化関数
def normalize_team_name(name):
    # 必要に応じてチーム名の正規化処理を追加
    name = name.strip()
    return name

# データベースに接続
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)
cursor = conn.cursor()

# シーズン情報を取得
cursor.execute("SELECT season_id, year FROM Seasons")
season_data = cursor.fetchall()
year_to_season_id = {year: season_id for season_id, year in season_data}

# ディレクトリ内のCSVファイルを取得
csv_files = [f for f in os.listdir(csv_directory) if f.startswith('standings_') and f.endswith('.csv')]

for filename in csv_files:
    print(filename)
    csv_path = os.path.join(csv_directory, filename)
    
    # ファイル名から年度とリーグを抽出
    match = re.search(r'standings_(\d{4})(\d{0,4})_(\w+)\.csv', filename)
    if match:
        year = int(match.group(1))
        # 日付が含まれる場合（YYYYMMDD）
        if match.group(2):
            month_day = match.group(2)
            date_str = f"{year}{month_day}"
            try:
                date = datetime.strptime(date_str, '%Y%m%d').date()
            except ValueError:
                print(f"日付の変換に失敗しました: {date_str} (ファイル: {filename})")
                continue
        else:
            # 日付がない場合は、その年のシーズン期間を使用
            date = None  # 後でシーズン開始日と終了日を取得して設定
        league_code = match.group(3)
        # リーグ名を設定
        if league_code.lower() == 'central':
            league = 'セントラル・リーグ'
        elif league_code.lower() == 'pacific':
            league = 'パシフィック・リーグ'
        else:
            league = None
            print(f"不明なリーグ: {league_code} (ファイル: {filename})")
            continue
    else:
        print(f"ファイル名から年度を取得できませんでした: {filename}")
        continue
    
    # シーズンIDを取得
    season_id = year_to_season_id.get(year)
    if not season_id:
        print(f"シーズンIDが見つかりませんでした: {year}")
        continue

    # シーズンの開始日と終了日を取得
    cursor.execute("SELECT start_date, end_date FROM Seasons WHERE season_id = %s", (season_id,))
    season_dates = cursor.fetchone()
    if season_dates:
        season_start_date, season_end_date = season_dates
    else:
        print(f"シーズンの日付情報が見つかりませんでした: {year}")
        continue

    # 日付が指定されていない場合はシーズンの中間日を使用
    if not date:
        date = season_start_date + (season_end_date - season_start_date) / 2

    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            team_name = normalize_team_name(row['チーム'])
            # チーム名と日付から team_id を取得
            cursor.execute("""
                SELECT team_id FROM team_history
                WHERE team_name = %s
                  AND %s BETWEEN start_date AND COALESCE(end_date, '9999-12-31')
                LIMIT 1
            """, (team_name, date))
            result = cursor.fetchone()
            if result:
                team_id = result[0]
            else:
                print(f"TeamHistoryからteam_idを取得できませんでした: {team_name} (ファイル: {filename}, 日付: {date})")
                continue

            try:
                games_played = int(row['試 合'])
                wins = int(row['勝 利'])
                losses = int(row['敗 北'])
                ties = int(row['引 分'])
                winning_percentage = float(row['勝 率'])
                games_behind = None
                # games_behind = row['ゲーム差']
                # if games_behind == '- -' or games_behind == '':
                #     games_behind = None
                # else:
                #     games_behind = float(games_behind)
            except ValueError as e:
                print(f"データの変換に失敗しました: {e} (ファイル: {filename}, チーム: {team_name})")
                continue

            # Standingsテーブルにデータを挿入
            insert_query = """
            INSERT INTO Standings (team_id, season_id, league, wins, losses, ties, games_played, winning_percentage, games_behind)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (team_id, season_id, league, wins, losses, ties, games_played, winning_percentage, games_behind)
            print(values)
            cursor.execute(insert_query, values)

# コミットしてデータベースに保存
conn.commit()

# 接続を閉じる
cursor.close()
conn.close()

print("データのインポートが完了しました。")


standings_2001_pacific.csv
(15, 52, 'パシフィック・リーグ', 78, 60, 2, 140, 0.565, None)
(10, 52, 'パシフィック・リーグ', 76, 63, 1, 140, 0.547, None)
(11, 52, 'パシフィック・リーグ', 73, 67, 0, 140, 0.521, None)
(9, 52, 'パシフィック・リーグ', 70, 66, 4, 140, 0.515, None)
(12, 52, 'パシフィック・リーグ', 64, 74, 2, 140, 0.464, None)
(13, 52, 'パシフィック・リーグ', 53, 84, 3, 140, 0.387, None)
standings_1967_pacific.csv
(9, 18, 'パシフィック・リーグ', 75, 55, 4, 134, 0.577, None)
(11, 18, 'パシフィック・リーグ', 66, 64, 10, 140, 0.508, None)
(13, 18, 'パシフィック・リーグ', 65, 65, 4, 134, 0.5, None)
(10, 18, 'パシフィック・リーグ', 64, 66, 3, 133, 0.492, None)
(12, 18, 'パシフィック・リーグ', 61, 69, 7, 137, 0.469, None)
(15, 18, 'パシフィック・リーグ', 59, 71, 2, 132, 0.454, None)
standings_1996_pacific.csv
(9, 47, 'パシフィック・リーグ', 74, 50, 6, 130, 0.597, None)
(13, 47, 'パシフィック・リーグ', 68, 58, 4, 130, 0.54, None)
(11, 47, 'パシフィック・リーグ', 62, 64, 4, 130, 0.492, None)
(15, 47, 'パシフィック・リーグ', 62, 67, 1, 130, 0.481, None)
(12, 47, 'パシフィック・リーグ', 60, 67, 3, 130, 0.472, None)
(10, 47, 'パシフィック・リーグ', 54, 74, 2, 130, 0